# Create merged NDC info file

2019-06-03

Want to put the full NDC with the rest of the metadata.

In [1]:
import pandas as pd

## Read simple package

In [2]:
package = pd.read_csv("../../pipeline/fda_ndc/simple_package.tsv", sep='\t')

In [3]:
package.shape

(246380, 4)

In [4]:
package.head()

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION
0,0002-0800_4bb5d1cb-0fa7-48c7-9f6d-8d45f9b91649,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...
1,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ..."
2,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ..."
3,0002-1407_14757f9d-f641-4836-acf3-229265588d1d,0002-1407,0002-1407-01,10 mL in 1 VIAL (0002-1407-01)
4,0002-1433_4468578a-47d2-488e-9fd4-a8322070392f,0002-1433,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...


## Read simple product

In [5]:
product = pd.read_csv("../../pipeline/fda_ndc/simple_product.tsv", sep='\t')

In [6]:
product.shape

(130033, 17)

In [7]:
product.head()

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,0002-0800_4bb5d1cb-0fa7-48c7-9f6d-8d45f9b91649,0002-0800,HUMAN OTC DRUG,Sterile Diluent,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,NDA,NDA018781,Eli Lilly and Company,WATER,1,mL/mL,NaN,NaN,N,20191231.0
1,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,NDA,NDA202008,Eli Lilly and Company,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20191231.0
2,0002-1407_14757f9d-f641-4836-acf3-229265588d1d,0002-1407,HUMAN PRESCRIPTION DRUG,Quinidine Gluconate,Quinidine Gluconate,SOLUTION,INTRAVENOUS,NDA,NDA007529,Eli Lilly and Company,QUINIDINE GLUCONATE,80,mg/mL,"Antiarrhythmic [EPC],Cytochrome P450 2D6 Inhib...",NaN,N,20191231.0
3,0002-1433_4468578a-47d2-488e-9fd4-a8322070392f,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,BLA,BLA125469,Eli Lilly and Company,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
4,0002-1434_4468578a-47d2-488e-9fd4-a8322070392f,0002-1434,HUMAN PRESCRIPTION DRUG,Trulicity,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,BLA,BLA125469,Eli Lilly and Company,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0


## Check that merge ids match up

In [8]:
packids = set(package["PRODUCTID"])

In [9]:
prodids = set(product["PRODUCTID"])

In [10]:
len(packids)

129764

In [11]:
len(prodids)

130033

In [12]:
packids <= prodids

False

This is weird... How can the ids be mismatched?

### Unknown package ids

In [13]:
len(packids - prodids)

4

In [14]:
(pd
    .Series(list(packids - prodids))
    .to_frame("PRODUCTID")
    .merge(package, how="left", on="PRODUCTID")
    .sort_values("NDCPACKAGECODE")
    .reset_index(drop=True)
)

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION
0,65862-171_e0f2459c-5edf-47f3-b8b2-dc699884d683,65862-171,65862-171-60,"60 TABLET, FILM COATED in 1 BOTTLE (65862-171-..."
1,65862-172_e0f2459c-5edf-47f3-b8b2-dc699884d683,65862-172,65862-172-60,"60 TABLET, FILM COATED in 1 BOTTLE (65862-172-..."
2,65862-173_e0f2459c-5edf-47f3-b8b2-dc699884d683,65862-173,65862-173-60,"60 TABLET, FILM COATED in 1 BOTTLE (65862-173-..."
3,65862-174_e0f2459c-5edf-47f3-b8b2-dc699884d683,65862-174,65862-174-60,"60 TABLET, FILM COATED in 1 BOTTLE (65862-174-..."


### Unknown product ids

In [15]:
len(prodids - packids)

273

In [16]:
(pd
    .Series(list(prodids - packids))
    .to_frame("PRODUCTID")
    .merge(product, how="left", on="PRODUCTID")
    .sort_values("PRODUCTNDC")
    .reset_index(drop=True)
).head(5)

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,0002-3004_f1404e9d-0c95-44e4-9eb0-7a2836a67bd8,0002-3004,HUMAN PRESCRIPTION DRUG,Prozac,Fluoxetine hydrochloride,"CAPSULE, DELAYED RELEASE",ORAL,NDA,NDA021235,Eli Lilly and Company,FLUOXETINE HYDROCHLORIDE,90,mg/1,"Serotonin Reuptake Inhibitor [EPC],Serotonin U...",NaN,N,20191231.0
1,0023-4385_f7c032a8-8330-4784-a5e7-f85dfec0cbe6,0023-4385,HUMAN PRESCRIPTION DRUG,BETAGAN,levobunolol hydrochloride,SOLUTION/ DROPS,OPHTHALMIC,NDA,NDA019219,"Allergan, Inc.",LEVOBUNOLOL HYDROCHLORIDE,5,mg/mL,"Adrenergic beta-Antagonists [MoA],beta-Adrener...",NaN,N,20201231.0
2,0054-0086_84e16e83-b683-4e03-908a-2732c8b59628,0054-0086,HUMAN PRESCRIPTION DRUG,Zolpidem Tartrate,zolpidem tartrate,"TABLET, FILM COATED",ORAL,ANDA,ANDA077214,"Roxane Laboratories, Inc",ZOLPIDEM TARTRATE,5,mg/1,"gamma-Aminobutyric Acid-ergic Agonist [EPC],GA...",CIV,N,20191231.0
3,0054-0087_84e16e83-b683-4e03-908a-2732c8b59628,0054-0087,HUMAN PRESCRIPTION DRUG,Zolpidem Tartrate,zolpidem tartrate,"TABLET, FILM COATED",ORAL,ANDA,ANDA077214,"Roxane Laboratories, Inc",ZOLPIDEM TARTRATE,10,mg/1,"gamma-Aminobutyric Acid-ergic Agonist [EPC],GA...",CIV,N,20191231.0
4,0054-0320_910d587e-5e7b-4a49-ae91-e802baf4bff0,0054-0320,HUMAN PRESCRIPTION DRUG,Temozolomide,Temozolomide,CAPSULE,ORAL,ANDA,ANDA202606,"Roxane Laboratories, Inc.",TEMOZOLOMIDE,5,mg/1,"Alkylating Activity [MoA],Alkylating Drug [EPC]",NaN,E,20181231.0


Ok, so it seems like the data is not super well formatted when released. Seems like this is all the super new products anyways, so we shouldn't need to worry.

---

## Merge as best as we can

In [17]:
res = package.merge(product, how="inner", on="PRODUCTID")

In [18]:
res.shape

(246376, 20)

In [19]:
res.head()

,PRODUCTID,PRODUCTNDC_x,NDCPACKAGECODE,PACKAGEDESCRIPTION,PRODUCTNDC_y,PRODUCTTYPENAME,PROPRIETARYNAME,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,0002-0800_4bb5d1cb-0fa7-48c7-9f6d-8d45f9b91649,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,0002-0800,HUMAN OTC DRUG,Sterile Diluent,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,NDA,NDA018781,Eli Lilly and Company,WATER,1,mL/mL,NaN,NaN,N,20191231.0
1,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,NDA,NDA202008,Eli Lilly and Company,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20191231.0
2,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,NDA,NDA202008,Eli Lilly and Company,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20191231.0
3,0002-1407_14757f9d-f641-4836-acf3-229265588d1d,0002-1407,0002-1407-01,10 mL in 1 VIAL (0002-1407-01),0002-1407,HUMAN PRESCRIPTION DRUG,Quinidine Gluconate,Quinidine Gluconate,SOLUTION,INTRAVENOUS,NDA,NDA007529,Eli Lilly and Company,QUINIDINE GLUCONATE,80,mg/mL,"Antiarrhythmic [EPC],Cytochrome P450 2D6 Inhib...",NaN,N,20191231.0
4,0002-1433_4468578a-47d2-488e-9fd4-a8322070392f,0002-1433,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,BLA,BLA125469,Eli Lilly and Company,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0


### Check things match

In [20]:
(res["PRODUCTNDC_x"] == res["PRODUCTNDC_y"]).all()

True

## Generate final table

In [21]:
final = (res
    .drop(["PRODUCTID", "PRODUCTNDC_y"], axis=1)
    .rename(columns={"PRODUCTNDC_x": "PRODUCTNDC"})
    .drop_duplicates()
)

In [22]:
final.shape

(246372, 18)

In [23]:
final.head()

,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,PRODUCTTYPENAME,PROPRIETARYNAME,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,HUMAN OTC DRUG,Sterile Diluent,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,NDA,NDA018781,Eli Lilly and Company,WATER,1,mL/mL,NaN,NaN,N,20191231.0
1,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",HUMAN PRESCRIPTION DRUG,Amyvid,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,NDA,NDA202008,Eli Lilly and Company,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20191231.0
2,0002-1200,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",HUMAN PRESCRIPTION DRUG,Amyvid,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,NDA,NDA202008,Eli Lilly and Company,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20191231.0
3,0002-1407,0002-1407-01,10 mL in 1 VIAL (0002-1407-01),HUMAN PRESCRIPTION DRUG,Quinidine Gluconate,Quinidine Gluconate,SOLUTION,INTRAVENOUS,NDA,NDA007529,Eli Lilly and Company,QUINIDINE GLUCONATE,80,mg/mL,"Antiarrhythmic [EPC],Cytochrome P450 2D6 Inhib...",NaN,N,20191231.0
4,0002-1433,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,HUMAN PRESCRIPTION DRUG,Trulicity,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,BLA,BLA125469,Eli Lilly and Company,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0


In [24]:
final["PRODUCTNDC"].nunique()

128080

In [25]:
final["NDCPACKAGECODE"].nunique()

246357

In [26]:
final.isnull().sum()

PRODUCTNDC                               0
NDCPACKAGECODE                           0
PACKAGEDESCRIPTION                       0
PRODUCTTYPENAME                          0
PROPRIETARYNAME                         11
NONPROPRIETARYNAME                       7
DOSAGEFORMNAME                           0
ROUTENAME                             2571
MARKETINGCATEGORYNAME                    0
APPLICATIONNUMBER                    34413
LABELERNAME                              0
SUBSTANCENAME                         3113
ACTIVE_NUMERATOR_STRENGTH             3114
ACTIVE_INGRED_UNIT                    3113
PHARM_CLASSES                       121349
DEASCHEDULE                         233995
NDC_EXCLUDE_FLAG                         0
LISTING_RECORD_CERTIFIED_THROUGH      5285
dtype: int64

In [27]:
final["NDC_EXCLUDE_FLAG"].value_counts()

N    183541
E     62831
Name: NDC_EXCLUDE_FLAG, dtype: int64

## Save to file

In [28]:
final.to_csv("../../pipeline/fda_ndc/ndc_info.tsv", sep='\t', index=False)